In [262]:
import os
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Flatten, Conv1D, Input, Concatenate, BatchNormalization, MaxPooling1D
from keras.layers.embeddings import Embedding

In [263]:
## Data
train_csv = "assets/train.csv"
df = pd.read_csv(train_csv)

In [282]:
## Labels
## labels = np.array(df.values[:,0].tolist())
labels = np.array([random.choice([0, 1, 2, 3, 4, 5, 6]) for x in range(df.shape[0])])

[4 1 3 1 0 2 6 2 2 3 6 6 0 6 6 2 5 5 4 3 3 4 0 0 2 4 6 4 2 1 6 3 6 3 1 3 2
 4 1 1 3 5 6 3 6 0 0 0 6 5 0 6 5 6 0 2 1 6 6 6 1 5 2 3 6 6 4 0 4 4 1 4 1 2
 2 6 4 0 6 3 4 5 6 0 6 1 2 3 1 1 3 4 5 0 3 6 3 0 3 1 5 5 6 6 2 2 2 4 3 0 0
 0 3 3 4 4 4 2 2 4 3 1 4 3 2 5 1 4 1 5 6 4 2 6 3 0 0 2 3 6 2 3 3 3 3 0 3 2
 4 1 0 2 6 3 0 0 4 1 2 6 3 4 0 0 6 4 1 6 5 4 5 0 0 0 1 5 4 0 3 3 1 1 2 2 3
 2 0 0 3 6 4 5 1 5 4 6 2 0 4 5 1 1 0 6 2 1 3 6 3 6 6 1 0 2 4 2 4 3 0 6 1 3
 2 5 6 5 0 0 3 3 1 6 2 5 2 4 1 6 6 4 5 6 5 1 2 0 0 5 5 0 0 4 4 1 4 3 1 1 0
 4 2 0 3 5 6 3 2 3 6 1 2 0 5 2 3 1 4 3 3 1 0 3 6 6 2 3 0 4 4 5 1 4 6 1 1 6
 3 4 3 5 4 2 4 2 3 3 2 5 1 6 1 2 3 4 5 4 5 5 4 0 1 5 4 4 4 1 0 4 6 3 4 2 1
 3 0 5 4 2 4 6 1 1 3 1 4 1 1 1 5 5 2 0 2 5 5 0 4 6 3 4 3 4 6 5 6 1 6 4 3 2
 2 3 1 2 4 1 1 0 6 3 5 5 6 6 4 0 0 2 6 3 0 6 5 6 3 6 4 6 6 5 2 1 6 2 2 5 1
 1 3 0 5 2 0 1 1 6 6 2 5 2 3 0 1 0 3 2 2 2 5 2 5 1 4 2 5 2 0 5 6 6 0 0 2 3
 0 4 0 2 0 3 2 6 0 0 4 1 2 5 3 3 5 2 1 1 1 5 4 4 5 6 0 5 6 1 1 1 5 0 3 1 3
 6 3 5 3 5 2 3 4 4 0 4 5 

In [266]:
lines = df.values[:,2].tolist()
raw_text = "".join(lines)

chars = sorted(list(set(raw_text)))
mapping = dict((c, i + 1) for i, c in enumerate(chars))

sequences = []
for line in lines:
    encoded_seq = [mapping.get(char) for char in line]
    sequences.append(encoded_seq)

vocab_size = len(mapping) + 1
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
print("Vob: %d" % vocab_size)
print("Len: %d" % max_length)

Vob: 334
Len: 537


In [283]:
inputs = Input(shape=(max_length, ))

embedding = Embedding(vocab_size, 16, input_length=max_length, trainable=False)(inputs)
convs = []
for n, fsz in [(128, 2), (192, 3), (256, 4), (512, 5)]:
    model = Conv1D(n, fsz, activation='relu', padding='same')(embedding)
    model = BatchNormalization()(model)
    model = Conv1D(n, fsz, activation='relu', padding='same')(model)
    model = MaxPooling1D()(model)
    convs.append(model)

model = Concatenate(axis=-1)(convs)
model = Flatten()(model)

model = BatchNormalization()(model)
model = Dense(7, activation='relu')(model)
model = BatchNormalization()(model)
model = Dense(7, activation='relu')(model)
model = BatchNormalization()(model)

model = Dense(6, activation='sigmoid')(model)
model = Model(inputs, model)
print(model.summary())

Model: "model_50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_82 (InputLayer)           (None, 537)          0                                            
__________________________________________________________________________________________________
embedding_112 (Embedding)       (None, 537, 16)      5344        input_82[0][0]                   
__________________________________________________________________________________________________
conv1d_308 (Conv1D)             (None, 537, 128)     4224        embedding_112[0][0]              
__________________________________________________________________________________________________
conv1d_310 (Conv1D)             (None, 537, 192)     9408        embedding_112[0][0]              
___________________________________________________________________________________________

In [284]:
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

In [285]:
model.fit(padded_sequences, labels, epochs=2, verbose=1)

ValueError: Error when checking target: expected dense_142 to have shape (6,) but got array with shape (1,)